In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import linear_model
import xgboost as xgb
from functools import reduce
%matplotlib inline

In [14]:
df=pd.read_csv(r"D:\ML\xgboost_test\customer_churn_dataset.csv")

In [16]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [17]:
df.columns

Index(['Customer ID', 'demo_count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Dependents', 'Tenure in Months', 'Phone Service', 'Multiple Lines',
       'Internet Service', 'Online Security', 'Online Backup',
       'Device Protection Plan', 'Premium Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charge', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')